# Global Power Rankings Hackathon


## Intro 
Hi my name is Giuseppe Frigeni and my nickname on discord is Camp. I started playing league in season 2 and i have been a fan of the pro scene since (i still remember watching season 2 Worlds and how hype it felt) even after I stopped playing the game some years ago. To this day i still watch the pro scene when i can and im watching daily this Worlds.

I recently started studying Data science and Machine Learning and I already had in mind a project for LoL when i stumbled on this hackaton. The project I had in mind is to create something similar to what they have in chess that in real time tells you who is winning using computer vision but i think is a bit too big and computing needy for a solo project.

After looking at this hackaton my thinking is to create an ELO system for the players. The ELO of a team is the average of the players and players gain or loose ELO on wins or losses.

I am not gonna use the stats of individual performance for two reasons: make the code faster without having to pull up the data for every single game (in tournament.json i can get everything i need) and because you cant say who performed better or worse just by looking at a scoreline or for an epic kill. Examples could be the Dyrus treatment where he would get camped during the game and have negative scorelines and then the team would win, should he get less ELO because his scoreline is bad? I think is very hard getting a factual insight in someone performance without looking at the game unfolding.

## ELO_players and problems

In [10]:
import requests
import json
import gzip
import shutil
import time
import os
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

To download the files i just used the code provided by you

In [2]:
S3_BUCKET_URL = "https://power-rankings-dataset-gprhack.s3.us-west-2.amazonaws.com"


def download_gzip_and_write_to_json(file_name):
   # If file already exists locally do not re-download game
   if os.path.isfile(f"{file_name}.json"):
       return

   response = requests.get(f"{S3_BUCKET_URL}/{file_name}.json.gz")
   if response.status_code == 200:
       try:
           gzip_bytes = BytesIO(response.content)
           with gzip.GzipFile(fileobj=gzip_bytes, mode="rb") as gzipped_file:
               with open(f"{file_name}.json", 'wb') as output_file:
                   shutil.copyfileobj(gzipped_file, output_file)
               print(f"{file_name}.json written")
       except Exception as e:
           print("Error:", e)
   else:
       print(f"Failed to download {file_name}")


def download_esports_files():
   directory = "esports-data"
   if not os.path.exists(directory):
       os.makedirs(directory)

   esports_data_files = ["leagues", "tournaments", "players", "teams", "mapping_data","tournaments_without_game_data","unfiltered_players","unfiltered_teams"]
   for file_name in esports_data_files:
       download_gzip_and_write_to_json(f"{directory}/{file_name}")


if __name__ == "__main__":
    download_esports_files()

esports-data/leagues.json written
esports-data/tournaments.json written
esports-data/players.json written
esports-data/teams.json written
esports-data/mapping_data.json written
esports-data/tournaments_without_game_data.json written
esports-data/unfiltered_players.json written
esports-data/unfiltered_teams.json written


First problem using pandas read_json the ids are too long and become wrong probably because of some approximation done by the command on large numbers so i used this workaround:

In [3]:
def json_to_df(path):
    with open(path, 'r') as j:
        contents = json.loads(j.read())

    df = pd.json_normalize(contents)
    return df

In [8]:
tournaments = json_to_df('esports-data/tournaments.json')
tournaments2 = json_to_df('esports-data/tournaments_without_game_data.json')
players = json_to_df('esports-data/unfiltered_players.json')
#unfiltered_players = json_to_df('esports-data/unfiltered_players.json')
leagues = json_to_df('esports-data/leagues.json')
teams = json_to_df('esports-data/unfiltered_teams.json')
mapping_data = json_to_df('esports-data/mapping_data.json')
new_leagues = pd.read_csv('new_leagues.csv')
new_leagues = new_leagues.drop('Unnamed: 0', axis = 1)
tournaments = pd.concat([tournaments, tournaments2], ignore_index=True)
tournaments.sort_values(by =['startDate'], inplace =True)
players = players.drop_duplicates(subset=['player_id'])
new_leagues.sort_values(by =['ELO'], ascending = False, inplace =True)

new_leagues is a the leagues dataframe modified by hand by me where i added a starting ELO for specific league.

In [9]:
new_leagues

,id,slug,ELO
16,98767975604431411,world,1600
15,98767991325878492,msi,1600
11,98767991314006698,lpl,1500
5,98767991310872058,lck,1500
36,123,eu,1400
8,98767991302996019,lec,1400
0,98767991299243165,lcs,1300
32,98767991335774713,lck_challengers_league,1200
12,104366947889790212,pcs,1200
7,105709090213554609,lco,1100


This is gonna be my main dataframe where i store the ELO of every player

In [11]:
ELO_players = pd.DataFrame(columns = ['id', 'handle', 'ELO'])
ELO_players

,id,handle,ELO


This is the main function to which i give a date and is gonna process all the tournaments starting with that date:

In [15]:
def advance_elo(date, ELO_players, new_leagues):
    for index, tournament in tournaments.iterrows():
        start_date = tournament.get("startDate", "")
        if start_date.startswith(date):
            if int(tournament['leagueId']) not in new_leagues.values:
                print(tournament['slug'] ,' not in leagues')
                continue
            print(f"Processing {tournament['slug']}")
            for stage in tournament["stages"]:
                for section in stage["sections"]:
                    for match in section["matches"]:
                        for game in match["games"]:
                            if game['id'] not in mapping_data.values:
                                    continue
                            if game["state"] == "completed":
                                elo_blue_side, elo_red_side, players_blue, players_red ,ELO_players = get_elo(game['id'], tournament['leagueId'], ELO_players,new_leagues)
                                # function to get the elo of both sides and the players playing in this game
                                k = 30 + 20 * (match['strategy']['count'] > 4) 
                                # coefficient for the game that is normally 30 but 50 for BO5 that normally imply more important games
                                result_blue_side = game['teams'][0]['result']['outcome'] # result of blue side
                                elo_change_blue, elo_change_red = elo_calc(elo_blue_side, elo_red_side, k, result_blue_side, stage['name']) 
                                #function to calculate the change in elo for both sides
                                ELO_players = elo_change(elo_change_blue, elo_change_red, ELO_players, players_blue, players_red, elo_blue_side, elo_red_side)
                                #function to change the elo of every single player
    
    return ELO_players

In [14]:
def get_elo(game_id,league_id, ELO_players,new_leagues):
    elo_blue_side = 0
    elo_red_side = 0
    players_blue = []
    players_red = []
    for i in range(1, 6, 1):
        participant_mapping = 'participantMapping.{}'.format(i)
        player_id = mapping_data.loc[mapping_data['esportsGameId'] == game_id][participant_mapping].item()
        if player_id not in players.values:
            continue
        players_blue.append(player_id)
        if player_id not in ELO_players.values :
            ELO_players = start_elo(player_id, league_id, ELO_players, new_leagues)
        elo_blue_side += ELO_players.loc[ELO_players['id'] == player_id]['ELO'].item()
    for i in range(6, 11, 1):
        participant_mapping = 'participantMapping.{}'.format(i)
        player_id = mapping_data.loc[mapping_data['esportsGameId'] == game_id][participant_mapping].item()
        if player_id not in players.values:
            continue
        players_red.append(player_id)
        if player_id not in ELO_players.values :
            ELO_players =start_elo(player_id, league_id, ELO_players, new_leagues)
        elo_red_side += ELO_players.loc[ELO_players['id'] == player_id]['ELO'].item()
    if len(players_blue) == 0:
        len_players_blue = 1
    else: 
        len_players_blue = len(players_blue)
    if len(players_red) == 0:
        len_players_red = 1
    else: 
        len_players_red = len(players_red)
    
    return elo_blue_side/len_players_blue, elo_red_side/len_players_red, players_blue, players_red, ELO_players

## ELO_teams e la riuscita

## ELO history and my data

## Conclusioni 